In [1]:
import requests
import urllib
import json
import pandas as pd

In [2]:
tokens = pd.read_csv('Twitter-Keys_4.20.21.txt', sep = '\t', header = None).T
tokens.columns = tokens.iloc[0]
tokens = tokens[1:]
#tokens.Bearer_Token[1]

In [3]:
def auth():
    return tokens.Bearer_Token[1]

#print(auth())

In [4]:
def create_url():
    #query = '("Columbus Crew") OR #Crew96 -is:retweet'
    query = urllib.parse.quote('("Columbus Crew") OR #Crew96 lang:en -is:retweet')
    tweet_fields = 'tweet.fields=author_id,conversation_id,public_metrics,id,text'
    url = 'https://api.twitter.com/2/tweets/search/recent?query={}&{}'.format(query, tweet_fields)
    return url

print(create_url())

https://api.twitter.com/2/tweets/search/recent?query=%28%22Columbus%20Crew%22%29%20OR%20%23Crew96%20lang%3Aen%20-is%3Aretweet&tweet.fields=author_id,conversation_id,public_metrics,id,text


In [5]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

#print(create_headers(tokens.Bearer_Token[1]))

In [6]:
def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [7]:
def one_request():
    bearer_token = auth()
    url = create_url()
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    #print(json.dumps(json_response, indent = 4, sort_keys = True))
    return json_response

In [8]:
results = one_request()

200


In [9]:
#results

In [10]:
results.keys()

dict_keys(['data', 'meta'])

In [11]:
tweet_dat = pd.DataFrame(results['data'])

In [12]:
tweet_dat.head(2)

,text,public_metrics,conversation_id,author_id,id
0,I'm gonna be a #columbus citizen this Friday. ...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1385085183213383680,868209115,1385085183213383680
1,I think a thing that will help to ease LigaMX ...,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1385084124092903429,3296109765,1385084124092903429


In [13]:
metrics = pd.DataFrame(list(tweet_dat['public_metrics']))

In [14]:
metrics.head(2)

,retweet_count,reply_count,like_count,quote_count
0,0,0,1,0
1,0,1,0,0


In [15]:
tweet_dat.drop('public_metrics', axis=1).head(2)

,text,conversation_id,author_id,id
0,I'm gonna be a #columbus citizen this Friday. ...,1385085183213383680,868209115,1385085183213383680
1,I think a thing that will help to ease LigaMX ...,1385084124092903429,3296109765,1385084124092903429


In [16]:
full_dat = pd.concat([tweet_dat.drop('public_metrics', axis = 1).reset_index(drop=True), metrics], axis=1)

In [17]:
full_dat.head(2)

,text,conversation_id,author_id,id,retweet_count,reply_count,like_count,quote_count
0,I'm gonna be a #columbus citizen this Friday. ...,1385085183213383680,868209115,1385085183213383680,0,0,1,0
1,I think a thing that will help to ease LigaMX ...,1385084124092903429,3296109765,1385084124092903429,0,1,0,0


In [18]:
results['meta']

{'newest_id': '1385085183213383680',
 'oldest_id': '1385045506670346249',
 'result_count': 10,
 'next_token': 'b26v89c19zqg8o3fostsvxkmztntx2w2gwhrpigv70nel'}

In [19]:
results['meta']['next_token']

'b26v89c19zqg8o3fostsvxkmztntx2w2gwhrpigv70nel'

Check out the [Pagination documentation](https://developer.twitter.com/en/docs/twitter-api/pagination)

In [20]:
def create_url_pag(next_token = None):
    if (next_token):
        #query = '("Columbus Crew") OR #Crew96 -is:retweet'
        query = urllib.parse.quote('("Columbus Crew") OR #Crew96 -is:retweet lang:en')
        tweet_fields = 'tweet.fields=author_id,conversation_id,public_metrics,id,text'
        pagination = str(next_token)
        url = 'https://api.twitter.com/2/tweets/search/recent?query={}&max_results=10&next_token={}&{}'.format(query, pagination, tweet_fields)
        return url
    else:
        #query = '("Columbus Crew") OR #Crew96 -is:retweet'
        query = urllib.parse.quote('("Columbus Crew") OR #Crew96 -is:retweet lang:en')
        tweet_fields = 'tweet.fields=author_id,conversation_id,public_metrics,id,text'
        url = 'https://api.twitter.com/2/tweets/search/recent?query={}&{}&max_results=10'.format(query, tweet_fields)
        return url
    
print(create_url_pag(None))
print(create_url_pag('000000000000000'))

https://api.twitter.com/2/tweets/search/recent?query=%28%22Columbus%20Crew%22%29%20OR%20%23Crew96%20-is%3Aretweet%20lang%3Aen&tweet.fields=author_id,conversation_id,public_metrics,id,text&max_results=10
https://api.twitter.com/2/tweets/search/recent?query=%28%22Columbus%20Crew%22%29%20OR%20%23Crew96%20-is%3Aretweet%20lang%3Aen&max_results=10&next_token=000000000000000&tweet.fields=author_id,conversation_id,public_metrics,id,text


In [21]:
def pages():
    count = 0
    next_token = None
    dat = pd.DataFrame()
    while (count < 3):
        bearer_token = auth()
        url = create_url_pag(next_token)
        #print(url)
        headers = create_headers(bearer_token)
        json_response = connect_to_endpoint(url, headers)
        #print(json.dumps(json_response, indent = 4, sort_keys = True))
        next_token = json_response['meta']['next_token']
        dat = dat.append(pd.DataFrame(json_response['data']))
        #print(pd.DataFrame(json_response['data']))
        count = count + 1
    return dat

In [22]:
pages = pages()

200
200
200


In [23]:
pages.head(2)

,public_metrics,author_id,text,id,conversation_id
0,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",868209115,I'm gonna be a #columbus citizen this Friday. ...,1385085183213383680,1385085183213383680
1,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",3296109765,I think a thing that will help to ease LigaMX ...,1385084124092903429,1385084124092903429


In [24]:
metrics_full = pd.DataFrame(list(pages['public_metrics']))

In [25]:
metrics_full.head(2)

,retweet_count,reply_count,like_count,quote_count
0,0,0,1,0
1,0,1,0,0


In [26]:
full_dat2 = pd.concat([pages.drop('public_metrics', axis = 1).reset_index(drop=True), metrics_full], axis=1)

In [27]:
full_dat2.head(2)

,author_id,text,id,conversation_id,retweet_count,reply_count,like_count,quote_count
0,868209115,I'm gonna be a #columbus citizen this Friday. ...,1385085183213383680,1385085183213383680,0,0,1,0
1,3296109765,I think a thing that will help to ease LigaMX ...,1385084124092903429,1385084124092903429,0,1,0,0


In [28]:
len(full_dat2.index)

30